In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/1900-01-01-2022-05-18-PRIO-GRID-ID.csv')

In [ ]:
!pip install transformers==3.1.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 884 kB 23.8 MB/s 
     |████████████████████████████████| 3.0 MB 69.3 MB/s 
     |████████████████████████████████| 1.3 MB 60.0 MB/s 
     |████████████████████████████████| 880 kB 80.7 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=8e0830aec73686a91fc59d42dc8e101d1f3071f7ae4e5ecd76817978405b828c
  Stored in directory: /root/.cache/pip/wheels/82/ab/9b/c15899bf659ba74f623ac776e861cf2eb8608c1825ddec66a4
Successfully built sacremoses


In [ ]:
from transformers import pipeline

In [ ]:
classifier = pipeline("zero-shot-classification", model ="facebook/bart-large-mnli", device=0)
# classifier = pipeline("zero-shot-classification", device=0) # to utilize GPU

Downloading:   0%|          | 0.00/908 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/899k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

Some weights of the model checkpoint at facebook/bart-large-mnli were not used when initializing BartForSequenceClassification: ['model.encoder.version', 'model.decoder.version']
- This IS expected if you are initializing BartForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BartForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#  classifier(sequence, candidate_labels, multi_class=True)

{'labels': ['politics', 'elections', 'public health', 'economics'],
 'scores': [0.972069501876831,
  0.967610776424408,
  0.03248710557818413,
  0.0061644683592021465],
 'sequence': 'Who are you voting for in 2020?'}

The second example is a bit harder. Let's see if we can improve the results by using a hypothesis template which is more specific to the setting of review sentiment analysis. Instead of the default, `This example is {}.`, we'll use, `The sentiment of this review is {}.` (where `{}` is replaced with the candidate class name)

In [ ]:
# sequences = [
#     "I hated this movie. The acting sucked.",
#     "This movie didn't quite live up to my high expectations, but overall I still really enjoyed it."
# ]
# candidate_labels = ["positive", "negative"]
# hypothesis_template = "The sentiment of this review is {}."

# classifier(sequences, candidate_labels, hypothesis_template=hypothesis_template)

[{'labels': ['negative', 'positive'],
  'scores': [0.9890093207359314, 0.010990672744810581],
  'sequence': 'I hated this movie. The acting sucked.'},
 {'labels': ['positive', 'negative'],
  'scores': [0.9581228494644165, 0.0418771356344223],
  'sequence': "This movie didn't quite live up to my high expectations, but overall I still really enjoyed it."}]

In [ ]:
candidate_labels = ["migration", "labor issues", "housing crisis", "terrorism", "land seizure", "death of important figure", 
                    "militant violence"]
hypothesis_template = "The theme in this article is {}."

In [ ]:
def classify(text):
  res = classifier(text, candidate_labels, hypothesis_template=hypothesis_template,multi_class=False)
  themes = []
  for i in range(0, 3):
    if(res['scores'][i] > 0.3):
      themes.append(res['labels'][i])
  return themes

In [ ]:
result = pd.DataFrame()
for i in range(0, len(df), 100):
  temp_df = df.copy()[i:i+100]
  temp_df['themes'] = temp_df['notes'].apply(lambda x: classify(x))
  result = pd.concat([result, temp_df])
  if(len(result) == 20000):
    result.to_csv('res_'+str(i)+'.csv')
    result = pd.DataFrame()

TypeError: ignored